<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Building-&amp;-training-the-model" data-toc-modified-id="Building-&amp;-training-the-model-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Building &amp; training the model</a></span><ul class="toc-item"><li><span><a href="#DataLoaders" data-toc-modified-id="DataLoaders-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>DataLoaders</a></span></li><li><span><a href="#Here-on-down-I'm-exploring-how-my-custom-loss_func-must-be-formatted." data-toc-modified-id="Here-on-down-I'm-exploring-how-my-custom-loss_func-must-be-formatted.-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Here on down I'm exploring how my custom loss_func must be formatted.</a></span></li></ul></li></ul></div>

# Building & training the model

In [ ]:
from fastai.vision.all import *


### Params ###
im_size      = 224
batch_size   = 64
path         = Path('/home/rory/data/coco2017')


### Load singles pickle ###
# load df
df = pd.read_pickle(path/'singles.pkl')
def col_to_L(df, col): return L((ColReader(col)(df)).to_list())
imp  = col_to_L(df, 'im')
bbox = col_to_L(df, 'bbox')
lbl  = col_to_L(df, 'lbl')
# create getters
imp2lbl  = {p:l for p,l in zip(imp,lbl)}
imp2bbox = {p:b for p,b in zip(imp,bbox)}
def get_lbl(p):  return imp2lbl[p]
def get_bbox(p): return imp2bbox[p]

## DataLoaders

In [ ]:
TensorBBox.create(get_bbox(imp[0]))

TensorBBox([[240.4500, 412.3600, 375.0000, 494.3800]])

In [ ]:
### Datasets & DataLoaders ###
# dss for im,bb,lbl
dss_tfms = [[PILImage.create],
            [get_bbox, TensorBBox.create],
            [get_lbl, Categorize()]]
splits = RandomSplitter(.15)(imp)
dss = Datasets(imp, tfms=dss_tfms, splits=splits)
dss.train[0]

RuntimeError: shape '[-1, 4]' is invalid for input of size 1

In [ ]:
# dls
cpu_tfms = [BBoxLabeler(), PointScaler(),
            Resize(im_size, method=ResizeMethod.Squish), ToTensor()]
gpu_tfms = [IntToFloatTensor(), Normalize.from_stats(*imagenet_stats)]
dls = dss.dataloaders(bs=64, after_item=cpu_tfms, after_batch=gpu_tfms, n_inp=1)

x,y,z = dls.one_batch()
x.shape, y.shape, z.shape

(torch.Size([64, 3, 224, 224]), torch.Size([64, 1, 4]), torch.Size([64]))

In [ ]:
### Datasets & DataLoaders ###
# dss for im,labeledbbox
categorize = Categorize(lbl_subset)
def get_labeledbbox(p):
    return LabeledBBox(TensorBBox.create(get_bbox(p)), categorize(get_lbl(p)))
dss_tfms = [[PILImage.create], [get_labeledbbox]]
splits = RandomSplitter(.15)(files_subset)
dss = Datasets(files_subset, tfms=dss_tfms, splits=splits)

fastai.vision.core.LabeledBBox

In [ ]:
# dls for im,labeledbbox
cpu_tfms = [PointScaler(), Resize(im_size, method=ResizeMethod.Squish), ToTensor()]
gpu_tfms = [IntToFloatTensor(), Normalize.from_stats(*imagenet_stats)]
dls = dss.dataloaders(bs=64, after_item=cpu_tfms, after_batch=gpu_tfms)

In [ ]:
x,y = dls.one_batch()
print(x.shape, y[0].shape, y[1].shape)

torch.Size([64, 3, 224, 224]) torch.Size([64, 1, 4]) torch.Size([64])


In [ ]:
### Model ###
class custom_module(Module):
    def __init__(self, body, head):
        self.body, self.head = body, head

    def forward(self, x):
        return self.head(self.body(x))
body = create_body(resnet34, pretrained=True)
head = create_head(1024, 4+len(categorize.vocab), ps=0.5)
mod = custom_module(body, head)

## Here on down I'm exploring how my custom loss_func must be formatted.

In [ ]:
# use this to check shape of y batch
learn = Learner(dls, mod, loss_func=MSELossFlat())
b = dls.one_batch()
learn._split(b)
learn.yb[0]

(#2) [TensorBBox([[[  1.2500,   0.0000, 640.0000, 480.0000]],

        [[129.4300, 165.3200, 164.7900, 203.8700]],

        [[ 49.9000,  62.3800, 640.0000, 229.3500]],

        [[  3.1600,   2.6400, 480.8300, 420.2000]],

        [[ 99.3000,  40.2600, 605.6600, 362.3200]],

        [[  8.6300, 122.2500, 392.6300, 336.5400]],

        [[410.4400, 125.1100, 516.0900, 238.0100]],

        [[ 30.6300, 111.5600, 404.1600, 562.6800]],

        [[166.1100, 103.5500, 463.8200, 429.3000]],

        [[229.5700, 108.3600, 577.6000, 258.9500]],

        [[216.8600,  85.4000, 492.2500, 421.2400]],

        [[138.0700, 363.8700, 430.0200, 631.3700]],

        [[ 99.9400, 238.2900, 159.7400, 305.7800]],

        [[ 26.9700,   0.0000, 640.0000, 474.6100]],

        [[159.3700,  87.9300, 329.6800, 319.7700]],

        [[180.6900, 144.1400, 366.8900, 195.5000]],

        [[ 90.0400,  38.0100, 409.8900, 591.1000]],

        [[236.2200, 156.2200, 408.8100, 335.2800]],

        [[288.3100, 127.5200, 320.57

In [ ]:
type(learn.yb)

tuple

In [ ]:

len(learn.yb)

1

In [ ]:
type(learn.yb[0])

fastai.vision.core.LabeledBBox

In [ ]:
learn.yb[0]

(#2) [TensorBBox([[[  1.2500,   0.0000, 640.0000, 480.0000]],

        [[129.4300, 165.3200, 164.7900, 203.8700]],

        [[ 49.9000,  62.3800, 640.0000, 229.3500]],

        [[  3.1600,   2.6400, 480.8300, 420.2000]],

        [[ 99.3000,  40.2600, 605.6600, 362.3200]],

        [[  8.6300, 122.2500, 392.6300, 336.5400]],

        [[410.4400, 125.1100, 516.0900, 238.0100]],

        [[ 30.6300, 111.5600, 404.1600, 562.6800]],

        [[166.1100, 103.5500, 463.8200, 429.3000]],

        [[229.5700, 108.3600, 577.6000, 258.9500]],

        [[216.8600,  85.4000, 492.2500, 421.2400]],

        [[138.0700, 363.8700, 430.0200, 631.3700]],

        [[ 99.9400, 238.2900, 159.7400, 305.7800]],

        [[ 26.9700,   0.0000, 640.0000, 474.6100]],

        [[159.3700,  87.9300, 329.6800, 319.7700]],

        [[180.6900, 144.1400, 366.8900, 195.5000]],

        [[ 90.0400,  38.0100, 409.8900, 591.1000]],

        [[236.2200, 156.2200, 408.8100, 335.2800]],

        [[288.3100, 127.5200, 320.57

In [ ]:
# from doc(Learner._split) – if n_inp=1, xb = b[0] and yb=b[1:].
    def _split(self, b):
        i = getattr(self.dls, 'n_inp', 1 if len(b)==1 else len(b)-1)
        self.xb,self.yb = b[:i],b[i:]

# from doc(Learner._do_one_batch). Notice that self.loss_func is passed (pred, *yb). My custom loss will conform.
    def _do_one_batch(self):
        self.pred = self.model(*self.xb)
        self('after_pred')
        if len(self.yb): self.loss = self.loss_func(self.pred, *self.yb) # here it is
        self('after_loss')
        if not self.training or not len(self.yb): return
        self('before_backward')
        self._backward()
        self('after_backward')
        self._step()
        self('after_step')
        self.opt.zero_grad()

I'm going to pause for now. I think the way forward is to make my loss function work for dss=im,bb,lbl; not im,lblbb.

In [ ]:
def rmse(p, yb):

def labeledbbox_loss(input, target):
    # RMSE
    ...
    # CEL
    ...

In [ ]:
### Train ###
lr_min, _ = learn.lr_find()
# learn.fit_one_cycle(10, lr=lr_min)
# learn.save(...)

def show_preds(dss=dss, inf=inf, pipe=pipe, n=8, offset=0, nrows=2, ncols=4, sz=224):
    ctxs = get_grid(n, nrows, ncols)
    for i,ctx in enumerate(ctxs):
        im,bb = pipe(dss[i+offset]) # tfms for resizing ims and bbs
        pred = inf.predict(im)
        show_image(im, ctx=ctx)
        ((bb+1)*sz//2).show(ctx=ctx)
        pred[0].show(ctx=ctx, color='magenta')
# p = Pipeline([PointScaler(), Resize(im_size, method=ResizeMethod.Squish)])
# inf = load_learner('_20201002_coco_tensorbox_learner_20201005.pkl')
# show_preds(dss, inf, p)